### Sydney Water Bubbler Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import requests
import json
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy import OpenCage
from sklearn.cluster import KMeans

%matplotlib inline

ModuleNotFoundError: No module named 'folium'

In [2]:
water_df = pd.read_csv('Drinking_fountains_water_bubblers.csv')
water_df.head()

,Longitude,Latitude,OBJECTID,site_name,Suburb,Location,Accessible
0,151.181929,-33.895707,1,WJ Thurbon Playground,Newtown,Park,No wheelchair access
1,151.192401,-33.879331,2,M J (Paddy) Dougherty Reserve,Glebe,Reserve,Wheelchair accessible
2,151.184712,-33.879971,3,St James Park,Glebe,Park,Wheelchair accessible
3,151.184337,-33.880301,4,St James Park,Glebe,Park,NaN
4,151.184114,-33.880468,5,John St Reserve,Glebe,Park,NaN


In [3]:
# Rename columns and drop unneeded ones

columns = ['Longitude', 'Latitude', 'Site', 'Suburb']
water_df.drop(['OBJECTID', 'Location', 'Accessible'], axis=1, inplace=True)
water_df.columns = columns
water_df.head()

,Longitude,Latitude,Site,Suburb
0,151.181929,-33.895707,WJ Thurbon Playground,Newtown
1,151.192401,-33.879331,M J (Paddy) Dougherty Reserve,Glebe
2,151.184712,-33.879971,St James Park,Glebe
3,151.184337,-33.880301,St James Park,Glebe
4,151.184114,-33.880468,John St Reserve,Glebe


In [4]:
print("Number of Bubblers: ", water_df.shape[0])

Number of Bubblers:  215


In [5]:
# Remove rows with nan values

water_df = water_df[water_df.Longitude.isna() == False]
water_df = water_df[water_df.Latitude.isna() == False]
water_df = water_df[water_df.Site.isna() == False]
water_df = water_df[water_df.Suburb.isna() == False]
water_df.shape

(215, 4)

In [6]:
water_df.count()

Longitude    215
Latitude     215
Site         215
Suburb       215
dtype: int64

In [7]:
water_df.dtypes

Longitude    float64
Latitude     float64
Site          object
Suburb        object
dtype: object

### Foursquare API Call

In [15]:
search_url = 'https://api.foursquare.com/v2/venues/search'
explore_url = 'https://api.foursquare.com/v2/venues/explore'

CLIENT_ID = '5EFTZ2TCXU5SKLZNSF3UFZ5JVMMULOSZQ5XBVD5BXP2C0GPA'
CLIENT_SECRET = 'ZOD0N4JU53LY5AE3LWK0ARSKH1QF3YRJWL0WEXL22C55BXCN'
VERSION = '20180605'

In [16]:
sample_bubbler = water_df.loc[0]
sample_bubbler.Suburb

'Newtown'

In [17]:
neighborhood_latitude = water_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = water_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = sample_bubbler.Suburb # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Newtown are -33.895707290374304, 151.18192926396196.


In [18]:
search_query = sample_bubbler.Suburb
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5EFTZ2TCXU5SKLZNSF3UFZ5JVMMULOSZQ5XBVD5BXP2C0GPA&client_secret=ZOD0N4JU53LY5AE3LWK0ARSKH1QF3YRJWL0WEXL22C55BXCN&ll=-33.895707290374304,151.18192926396196&v=20180605&query=Newtown&radius=500&limit=100'

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
bubbler_venues = getNearbyVenues(names=water_df['Site'],
                                   latitudes=water_df['Latitude'],
                                   longitudes=water_df['Longitude']
                                  )

WJ Thurbon Playground
M J (Paddy) Dougherty Reserve
St James Park
St James Park
John St Reserve
May Pitt Playground
Seamer St Reserve
Lyons Rd Reserve
Glebe Foreshore Walk Stage 5
Peace Park
Oxford Square
Frog Hollow Reserve
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Sydney Park
Pitt Street Mall
Pitt Street Mall
Pitt Street Mall
Pitt Street Mall
Munn Reserve
Amy Street Reserve
Amy Street Reserve
5010 O'Connell Street
Elizabeth McCrea Playground
5040 MacDonald Street
Wentworth Park
Wentworth Park
Wentworth Park
Wentworth Park
Wentworth Park
Wentworth Park
5010 Morrissey Road
Hyde Park South
Hyde Park South
Hyde Park South
Hyde Park South
Hyde Park South
Hyde Park South
5020 Rothschild Avenue
5010 Archibald Avenue
4010 King Street
4010 King Street
4090 Erskineville Road
4020 Barcom Avenue
4220 Cleveland Street
4060 George Street
Matron Ruby Park - 3 Joynton Avenue


In [21]:
print(bubbler_venues.shape)
bubbler_venues.head()

(11038, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,WJ Thurbon Playground,-33.895707,151.181929,Gelato Messina,-33.896060,151.181170,Ice Cream Shop
1,WJ Thurbon Playground,-33.895707,151.181929,Dendy Cinemas,-33.896094,151.180544,Indie Movie Theater
2,WJ Thurbon Playground,-33.895707,151.181929,Brewtown Newtown,-33.893849,151.182478,Café
3,WJ Thurbon Playground,-33.895707,151.181929,Delhi 'O' Delhi,-33.897142,151.180542,Indian Restaurant
4,WJ Thurbon Playground,-33.895707,151.181929,Black Sheep,-33.895758,151.181115,Cocktail Bar


In [22]:
bubbler_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
3000 Hay Street,100,100,100,100,100,100
4010 College Street,90,90,90,90,90,90
4010 King Street,103,103,103,103,103,103
4020 Barcom Avenue,100,100,100,100,100,100
4040 Kent Street,75,75,75,75,75,75
...,...,...,...,...,...,...
Waterloo Oval,64,64,64,64,64,64
Waterloo Park & Mount Carmel,20,20,20,20,20,20
Wattle & Broadway Rest Area,100,100,100,100,100,100


In [23]:
print('There are {} uniques categories.'.format(len(bubbler_venues['Venue Category'].unique())))

There are 249 uniques categories.


In [24]:
# one hot encoding
bubbler_onehot = pd.get_dummies(bubbler_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bubbler_onehot['Neighborhood'] = bubbler_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bubbler_onehot.columns[-1]] + list(bubbler_onehot.columns[:-1])
bubbler_onehot = bubbler_onehot[fixed_columns]

bubbler_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,WJ Thurbon Playground,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,WJ Thurbon Playground,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,WJ Thurbon Playground,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,WJ Thurbon Playground,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,WJ Thurbon Playground,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
bubbler_onehot.shape

(11038, 250)

In [26]:
bubbler_grouped = bubbler_onehot.groupby('Neighborhood').mean().reset_index()
bubbler_grouped

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,3000 Hay Street,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.010000,0.0,0.010000,0.0,0.01,0.000000,0.0,0.0,0.01
1,4010 College Street,0.011111,0.0,0.0,0.00,0.0,0.011111,0.011111,0.0,0.0,...,0.0,0.011111,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00
2,4010 King Street,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.019417,0.0,0.019417,0.0,0.00,0.019417,0.0,0.0,0.00
3,4020 Barcom Avenue,0.020000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.010000,0.0,0.0,0.01
4,4040 Kent Street,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Waterloo Oval,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00
146,Waterloo Park & Mount Carmel,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00
147,Wattle & Broadway Rest Area,0.000000,0.0,0.0,0.01,0.0,0.020000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.010000,0.0,0.00,0.020000,0.0,0.0,0.00
148,Wentworth Park,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00


In [27]:
bubbler_grouped.shape

(150, 250)

In [28]:
num_top_venues = 5

for hood in bubbler_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bubbler_grouped[bubbler_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----3000 Hay Street----
                 venue  freq
0      Thai Restaurant  0.16
1                 Café  0.13
2                Hotel  0.06
3  Japanese Restaurant  0.05
4          Coffee Shop  0.03


----4010 College Street----
                venue  freq
0                Café  0.18
1         Coffee Shop  0.04
2                Park  0.03
3              Museum  0.03
4  Italian Restaurant  0.03


----4010 King Street----
                venue  freq
0                Café  0.17
1     Thai Restaurant  0.10
2         Pizza Place  0.06
3         Coffee Shop  0.04
4  Italian Restaurant  0.04


----4020 Barcom Avenue----
                venue  freq
0                Café  0.27
1  Italian Restaurant  0.07
2                 Pub  0.05
3                 Bar  0.05
4         Pizza Place  0.03


----4040 Kent Street----
                venue  freq
0                Café  0.13
1                 Bar  0.08
2         Coffee Shop  0.08
3               Hotel  0.07
4  Italian Restaurant  0.05


----4060 Cowper

                   venue  freq
0        Harbor / Marina  0.21
1                   Park  0.21
2                 Hostel  0.11
3                   Café  0.11
4  Sri Lankan Restaurant  0.05


----Bowden Playground----
                venue  freq
0                Café  0.25
1  Italian Restaurant  0.11
2                 Pub  0.04
3              Bakery  0.04
4         Supermarket  0.04


----Burren Street Reserve 2----
                 venue  freq
0                  Pub  0.10
1  Japanese Restaurant  0.07
2            Bookstore  0.07
3                 Café  0.07
4    Indian Restaurant  0.03


----Charles Kernan Reserve----
                   venue  freq
0                   Café  0.29
1                    Pub  0.11
2            Coffee Shop  0.09
3            Beer Garden  0.06
4  Performing Arts Venue  0.06


----Chelsea St Playground----
               venue  freq
0               Café  0.21
1  Indian Restaurant  0.05
2                Pub  0.05
3   Greek Restaurant  0.05
4             Lounge  0.

4  Thai Restaurant  0.03


----Pinkstone Playground----
             venue  freq
0              Bar  0.10
1             Café  0.08
2  Thai Restaurant  0.05
3              Pub  0.05
4        Bookstore  0.05


----Pirrama Park----
                venue  freq
0                Café  0.19
1  Chinese Restaurant  0.09
2     Thai Restaurant  0.08
3                Park  0.08
4       Boat or Ferry  0.04


----Pitt Street Mall----
          venue  freq
0          Café  0.13
1   Coffee Shop  0.07
2  Cocktail Bar  0.06
3         Hotel  0.06
4           Bar  0.05


----Pottinger Park (West) The Paddock----
                   venue  freq
0                   Café  0.14
1                    Pub  0.07
2  Australian Restaurant  0.06
3                  Hotel  0.06
4             Restaurant  0.05


----Prince Alfred Park----
                 venue  freq
0                 Café  0.21
1          Coffee Shop  0.06
2                  Bar  0.06
3  Lebanese Restaurant  0.05
4         Cocktail Bar  0.03


----Recon

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bubbler_grouped['Neighborhood']

for ind in np.arange(bubbler_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bubbler_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3000 Hay Street,Thai Restaurant,Café,Hotel,Japanese Restaurant,Korean Restaurant,Sandwich Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Pizza Place
1,4010 College Street,Café,Coffee Shop,Italian Restaurant,Park,Museum,Shopping Mall,Thai Restaurant,Pub,Hotel,Clothing Store
2,4010 King Street,Café,Thai Restaurant,Pizza Place,Park,Bar,Coffee Shop,Burger Joint,Italian Restaurant,Pub,College Rec Center
3,4020 Barcom Avenue,Café,Italian Restaurant,Pub,Bar,Pizza Place,American Restaurant,Lounge,Gay Bar,Seafood Restaurant,Thai Restaurant
4,4040 Kent Street,Café,Coffee Shop,Bar,Hotel,Italian Restaurant,Speakeasy,Seafood Restaurant,Restaurant,Bakery,Pub


In [38]:
# set number of clusters
kclusters = 5

bubbler_grouped_clustering = bubbler_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bubbler_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 2, 2, 1, 2, 2, 2], dtype=int32)

In [55]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bubbler_merged = water_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bubbler_merged = bubbler_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Site')

bubbler_merged.head() # check the last columns!

,Longitude,Latitude,Site,Suburb,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,151.181929,-33.895707,WJ Thurbon Playground,Newtown,2,Café,Thai Restaurant,Bar,Deli / Bodega,Ice Cream Shop,Japanese Restaurant,Cocktail Bar,Pub,Bakery,Coffee Shop
1,151.192401,-33.879331,M J (Paddy) Dougherty Reserve,Glebe,2,Café,Bar,Italian Restaurant,Coffee Shop,Eastern European Restaurant,Burger Joint,Lebanese Restaurant,Dumpling Restaurant,Tapas Restaurant,Indian Restaurant
2,151.184712,-33.879971,St James Park,Glebe,1,Café,Pub,Pizza Place,Indian Restaurant,Japanese Restaurant,Bakery,Thai Restaurant,Eastern European Restaurant,Dumpling Restaurant,Italian Restaurant
3,151.184337,-33.880301,St James Park,Glebe,1,Café,Pub,Pizza Place,Indian Restaurant,Japanese Restaurant,Bakery,Thai Restaurant,Eastern European Restaurant,Dumpling Restaurant,Italian Restaurant
4,151.184114,-33.880468,John St Reserve,Glebe,1,Café,Pub,Pizza Place,Indian Restaurant,Thai Restaurant,Bakery,Japanese Restaurant,Park,Beer Garden,Seafood Restaurant


In [50]:
from geopy.geocoders import Nominatim 
address = 'Sydney, NSW'

geolocator = Nominatim(user_agent="Sydney_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sydney are -33.8888621, 151.2048978618509.


In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bubbler_merged['Latitude'], bubbler_merged['Longitude'], bubbler_merged['Site'], bubbler_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [67]:
bubbler_merged.loc[bubbler_merged['Cluster Labels'] == 0, bubbler_merged.columns[[2] + list(range(5, bubbler_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Glebe Foreshore Walk Stage 5,Seafood Restaurant,Café,Fish Market,Pub,Park,Pizza Place,Sushi Restaurant,Grocery Store,Bakery,Wine Bar
72,Mary O'Brien Reserve,Park,Café,Pizza Place,Supermarket,Thai Restaurant,Mexican Restaurant,Seafood Restaurant,Liquor Store,Bakery,Sandwich Place
73,Elizabeth Street Reserve,Café,Park,Supermarket,Italian Restaurant,Japanese Restaurant,Coffee Shop,Photography Studio,Bakery,Seafood Restaurant,Mexican Restaurant
103,Tote Park,Café,Park,Furniture / Home Store,Pizza Place,BBQ Joint,Dumpling Restaurant,Liquor Store,Supermarket,Rental Car Location,Gastropub
104,Tote Park,Café,Park,Furniture / Home Store,Pizza Place,BBQ Joint,Dumpling Restaurant,Liquor Store,Supermarket,Rental Car Location,Gastropub
105,Joynton Park,Café,Park,Pizza Place,Mexican Restaurant,BBQ Joint,Shopping Mall,Seafood Restaurant,Cantonese Restaurant,Sandwich Place,Furniture / Home Store
106,Joynton Park,Café,Park,Pizza Place,Mexican Restaurant,BBQ Joint,Shopping Mall,Seafood Restaurant,Cantonese Restaurant,Sandwich Place,Furniture / Home Store
108,Harold Park,Café,American Restaurant,Park,Breakfast Spot,Spanish Restaurant,Pub,Pizza Place,Pet Store,Fish & Chips Shop,Basketball Court
109,Nuffield Park,Café,Park,Pizza Place,Mexican Restaurant,BBQ Joint,Shopping Mall,Seafood Restaurant,Cantonese Restaurant,Sandwich Place,Supermarket
127,Federal Park 1,Park,Café,American Restaurant,Pet Store,Fish & Chips Shop,Middle Eastern Restaurant,Steakhouse,Fried Chicken Joint,Light Rail Station,Food Court


In [68]:
bubbler_merged.loc[bubbler_merged['Cluster Labels'] == 1, bubbler_merged.columns[[2] + list(range(5, bubbler_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St James Park,Café,Pub,Pizza Place,Indian Restaurant,Japanese Restaurant,Bakery,Thai Restaurant,Eastern European Restaurant,Dumpling Restaurant,Italian Restaurant
3,St James Park,Café,Pub,Pizza Place,Indian Restaurant,Japanese Restaurant,Bakery,Thai Restaurant,Eastern European Restaurant,Dumpling Restaurant,Italian Restaurant
4,John St Reserve,Café,Pub,Pizza Place,Indian Restaurant,Thai Restaurant,Bakery,Japanese Restaurant,Park,Beer Garden,Seafood Restaurant
5,May Pitt Playground,Café,Pub,Yoga Studio,Gym / Fitness Center,Motorcycle Shop,Sushi Restaurant,Thai Restaurant,Paper / Office Supplies Store,Pizza Place,Portuguese Restaurant
7,Lyons Rd Reserve,Café,Convenience Store,Brewery,Korean Restaurant,Liquor Store,Donut Shop,Sushi Restaurant,Thai Restaurant,Furniture / Home Store,Climbing Gym
...,...,...,...,...,...,...,...,...,...,...,...
201,Beare Park,Café,Park,Chinese Restaurant,Wine Bar,Japanese Restaurant,Australian Restaurant,Bookstore,Tennis Court,Fish & Chips Shop,Ramen Restaurant
203,Rushcutters Bay Park,Café,Harbor / Marina,Italian Restaurant,Bar,Park,Pizza Place,Australian Restaurant,Japanese Restaurant,Thai Restaurant,Sushi Restaurant
204,Rushcutters Bay Park,Café,Harbor / Marina,Italian Restaurant,Bar,Park,Pizza Place,Australian Restaurant,Japanese Restaurant,Thai Restaurant,Sushi Restaurant
210,Dr H J Foley Rest Park,Café,Pub,Bakery,Pizza Place,Indian Restaurant,Japanese Restaurant,Bar,Seafood Restaurant,Thai Restaurant,Eastern European Restaurant


In [69]:
bubbler_merged.loc[bubbler_merged['Cluster Labels'] == 2, bubbler_merged.columns[[2] + list(range(5, bubbler_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,WJ Thurbon Playground,Café,Thai Restaurant,Bar,Deli / Bodega,Ice Cream Shop,Japanese Restaurant,Cocktail Bar,Pub,Bakery,Coffee Shop
1,M J (Paddy) Dougherty Reserve,Café,Bar,Italian Restaurant,Coffee Shop,Eastern European Restaurant,Burger Joint,Lebanese Restaurant,Dumpling Restaurant,Tapas Restaurant,Indian Restaurant
6,Seamer St Reserve,Café,Bar,Indian Restaurant,Eastern European Restaurant,Pub,Pizza Place,Park,Tapas Restaurant,Mexican Restaurant,Japanese Restaurant
9,Peace Park,Café,Bar,Pub,Thai Restaurant,Burger Joint,Shopping Mall,Multiplex,Juice Bar,Italian Restaurant,Ice Cream Shop
10,Oxford Square,Café,Italian Restaurant,Japanese Restaurant,Pizza Place,Bar,Hotel,Wine Bar,Liquor Store,Cocktail Bar,Thai Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
199,Arthur McElhone Reserve,Café,Australian Restaurant,Italian Restaurant,Park,Japanese Restaurant,Trail,Thai Restaurant,Bar,Pub,Chinese Restaurant
202,Fitzroy Gardens,Café,Italian Restaurant,Australian Restaurant,Japanese Restaurant,Hotel,Pub,Coffee Shop,Thai Restaurant,Gym,Hostel
211,Thomas Portley Reserve,Seafood Restaurant,Café,Fish Market,Bakery,Farmers Market,Thai Restaurant,Sushi Restaurant,Bar,Park,Coffee Shop
212,Mary Ann Street Park,Café,Thai Restaurant,Coffee Shop,Pub,Taiwanese Restaurant,Bar,Hotel,Chinese Restaurant,Noodle House,Japanese Restaurant


In [70]:
bubbler_merged.loc[bubbler_merged['Cluster Labels'] == 3, bubbler_merged.columns[[2] + list(range(5, bubbler_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Southern Cross Dr Reserve,Supermarket,Golf Course,Hotel Bar,Grocery Store,Farmers Market,Shopping Mall,Park,Department Store,Dessert Shop,Flea Market
114,Kimberly Grove Reserve,Golf Course,Cosmetics Shop,Thai Restaurant,Café,Bakery,Deli / Bodega,Department Store,Food Service,Food Court,Flea Market
115,Bannerman Cres.Reserve,Golf Course,Park,Thai Restaurant,Food Truck,Food Court,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant


In [71]:
bubbler_merged.loc[bubbler_merged['Cluster Labels'] == 4, bubbler_merged.columns[[2] + list(range(5, bubbler_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Bowden Playground,Café,Italian Restaurant,Shoe Store,Furniture / Home Store,Gourmet Shop,Bakery,Park,BBQ Joint,Coffee Shop,Supermarket
71,Les White Reserve,Café,Italian Restaurant,Convenience Store,Coffee Shop,Supermarket,Bistro,Furniture / Home Store,Seafood Restaurant,Gourmet Shop,Bakery
78,Dibbs St Reserve,Café,Italian Restaurant,Pub,Japanese Restaurant,Seafood Restaurant,Park,BBQ Joint,Coffee Shop,Breakfast Spot,Supermarket
87,Solander Park,Pub,Café,Italian Restaurant,Bar,Gourmet Shop,Park,Train Station,Bistro,Seafood Restaurant,Thai Restaurant
102,Harry Noble Reserve,Café,Pub,Italian Restaurant,Convenience Store,Train Station,Men's Store,Seafood Restaurant,Furniture / Home Store,Supermarket,Bistro
137,Erskineville Park and Oval,Café,Italian Restaurant,Pub,Supermarket,Bistro,Bakery,Gourmet Shop,Liquor Store,Coffee Shop,Japanese Restaurant
138,Erskineville Park and Oval,Café,Italian Restaurant,Pub,Supermarket,Bistro,Bakery,Gourmet Shop,Liquor Store,Coffee Shop,Japanese Restaurant
197,5040 Renwick Street,Café,Park,Vietnamese Restaurant,Italian Restaurant,Pub,Coffee Shop,Bistro,Supermarket,Market,Office
